# Fix pathing

In [1]:
import sys


sys.path.append("../..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))))


# Imports

In [3]:
import plotter
import datahandler

import matplotlib.pyplot
import numpy as np
import pandas as pd
import seaborn as sns
import IPython.display
import json
import matplotlib.pyplot as plt


# Constants

In [4]:
FOLDER_NAMES = ["ex_unknown_least_NSGA2", "ex_unknown_avg_NSGA2", "ex_unknown_most_NSGA2"]
FILE_INFO = {
    "ex_unknown_least_NSGA2": ("Lowest Activity", "green"),

    "ex_unknown_avg_NSGA2": ("Average Activity", "blue"),

    "ex_unknown_most_NSGA2": ("Highest Activity", "red"),
}

FOLDER_PATH = os.path.join(os.path.dirname(constants.PROJECT_DIRECTORY_PATH), "Simulator", "data")

TIME_SEGMENTS = [1, 4]
SEEDS = list(range(10))

deep_palette = sns.color_palette("deep")
deep_colors = {
    "blue": deep_palette[0],
    "orange": deep_palette[1],
    "green": deep_palette[2],
    "red": deep_palette[3],
}

bright_palette = sns.color_palette("bright")
bright_colors = {
    "blue": bright_palette[0],
    "orange": bright_palette[1],
    "green": bright_palette[2],
    "red": bright_palette[3],
}


In [5]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(False, False, True)


Loading dataset: 100%|██████████| 2/2 [00:03<00:00,  1.52s/it]


# Methods

In [6]:
def load_json():
    dataset = {}

    for folder_name in FOLDER_NAMES:
        for ts in TIME_SEGMENTS:
            for seed in SEEDS:
                day, color = FILE_INFO[folder_name]

                with open(os.path.join(FOLDER_PATH, folder_name, f"heuristic_ts={ts}_seed={seed}.json"), 'r') as file:
                    dataset[(day, color, ts, seed)] = json.load(file)

    return dataset


In [7]:
def compute_metrics(dataset):
    dataframe = pd.DataFrame()

    dataframe["day"] = "None"
    dataframe["color"] = "None"
    dataframe["ts"] = 0
    dataframe["seed"] = 0
    dataframe["generation"] = 0
    dataframe["individual"] = 0
    dataframe["diversity"] = 0
    dataframe["front_number"] = np.nan
    dataframe["crowding_distance"] = np.nan
    dataframe["percentage_violations"] = 0
    dataframe["percentage_violations_urban"] = 0
    dataframe["percentage_violations_rural"] = 0
    dataframe["avg_response_time_urban_a"] = 0
    dataframe["avg_response_time_urban_h"] = 0
    dataframe["avg_response_time_urban_v1"] = 0
    dataframe["avg_response_time_rural_a"] = 0
    dataframe["avg_response_time_rural_h"] = 0
    dataframe["avg_response_time_rural_v1"] = 0

    for (day, color, ts, seed), data in dataset.items():
        generation = list(range(len(data['diversity'])))[-1]

        individuals = list(range(len(data['percentage_violations'][generation])))

        diversity = data['diversity'][generation][0]

        for individual in individuals:
            objectives = []
            objective_keys = [
                "front_number",
                "crowding_distance",
                "percentage_violations",
                "percentage_violations_urban",
                "percentage_violations_rural",
                "avg_response_time_urban_a",
                "avg_response_time_urban_h",
                "avg_response_time_urban_v1",
                "avg_response_time_rural_a",
                "avg_response_time_rural_h",
                "avg_response_time_rural_v1",
            ]

            for objective_key in objective_keys:
                if objective_key in data:
                    objective = data[objective_key][generation][individual]
                else:
                    objective = np.nan
                
                objectives.append(objective)

            # add row
            dataframe.loc[len(dataframe.index)] = [
                day,
                color,
                ts,
                seed,
                generation,
                individual,
                diversity,
                *objectives
            ]
    
    dataframe.drop_duplicates(
        subset=[
            "day",
            "ts",
            "seed",
            "generation",
            "diversity",
            "front_number",
            "percentage_violations",
            "percentage_violations_urban",
            "percentage_violations_rural",
            "avg_response_time_urban_a",
            "avg_response_time_urban_h",
            "avg_response_time_urban_v1",
            "avg_response_time_rural_a",
            "avg_response_time_rural_h",
            "avg_response_time_rural_v1",
        ],
        inplace=True
    )
    
    return dataframe


In [8]:
def load_csv(filename = "events"):
    dataframes = []
    for seed in list(range(10)):
        dataframes.append(pd.read_csv(os.path.join(FOLDER_PATH, filename + "_seed=" + str(seed) + ".csv")))
    
    dataframe = pd.concat(dataframes, ignore_index=True)

    response_time_cols = [
        'duration_incident_creation',
        'duration_resource_appointment',
        'duration_resource_preparing_departure',
        'duration_dispatching_to_scene'
    ]
    dataframe['total_response_time'] = dataframe[response_time_cols].sum(axis=1)

    dataframe["complied"] = np.nan
    # Define the criteria for response times
    criteria = {
        ('A', True): 12 * 60,
        ('A', False): 25 * 60,
        ('H', True): 30 * 60,
        ('H', False): 40 * 60
    }

    for (triage, urban), group in dataframe.groupby(['triage_impression_during_call', 'urban']):
        if triage == 'V1':
            continue
    
        limit = criteria.get((triage, urban))
        if limit is not None:
            dataframe.loc[group.index, 'complied'] = group['total_response_time'] < limit

    return dataframe


In [9]:
def print_info(dataframe: pd.DataFrame):
    filtered_df = dataframe.dropna(subset=['complied'])

    # compliance
    numerator = filtered_df['complied'].count()
    true_count = filtered_df['complied'].sum()

    compliance_rate = true_count / numerator

    print(f"Compliance Rate: {(abs(compliance_rate - 1) * 100):.2f}")

    # complaince (U)
    # compliance (R)
    compliance_rates = filtered_df.groupby('urban')['complied'].agg(lambda x: x.mean())

    groups = []
    rates = []

    for group, rate in compliance_rates.items():
        groups.append(group)
        rates.append(rate)
    
    for group, rate in zip(groups[::-1], rates[::-1]):
        print(f"Compliance Rate for {'Urban' if group else 'Non-Urban'}: {(abs(rate - 1) * 100):.2f}")


# Main

In [10]:
dataset = load_json()
dataframe = compute_metrics(dataset)

dataframe.info()


<class 'pandas.core.frame.DataFrame'>
Index: 106 entries, 0 to 1180
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   day                          106 non-null    object 
 1   color                        106 non-null    object 
 2   ts                           106 non-null    int64  
 3   seed                         106 non-null    int64  
 4   generation                   106 non-null    int64  
 5   individual                   106 non-null    int64  
 6   diversity                    106 non-null    float64
 7   front_number                 106 non-null    int64  
 8   crowding_distance            106 non-null    float64
 9   percentage_violations        106 non-null    float64
 10  percentage_violations_urban  106 non-null    float64
 11  percentage_violations_rural  106 non-null    float64
 12  avg_response_time_urban_a    106 non-null    float64
 13  avg_response_time_urban_

In [11]:
day = "Highest Activity"
ts = 4


In [12]:
indices = []

seed = 0
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 1
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 2
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 3
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 4
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 5
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 6
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 7
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 8
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 9
filtered_df = dataframe[
    (dataframe["day"] == day) &
    (dataframe["ts"] == ts) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["day"] == day) & (dataframe["ts"] == ts) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

print(indices)


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1000,Highest Activity,red,4,0,182,0,0.15,0,1.797690e+308,0.163934,0.16875,0.130435,628.230,1149.30,3719.82,1000.130,1633.38,4704.25
1002,Highest Activity,red,4,0,182,2,0.15,0,1.797690e+308,0.169399,0.19375,0.000000,648.230,1133.05,3720.39,909.400,1348.62,4490.38
1004,Highest Activity,red,4,0,182,4,0.15,0,1.775000e+00,0.174863,0.18750,0.086957,629.757,1156.17,3660.17,990.133,1779.75,4574.38


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1020,Highest Activity,red,4,1,210,0,0.15,0,1.797690e+308,0.153005,0.15625,0.130435,614.838,1192.62,3991.73,1042.800,1610.25,4570.25
1036,Highest Activity,red,4,1,210,16,0.15,0,4.464290e-01,0.158470,0.17500,0.043478,622.419,1235.50,3666.45,979.733,1620.12,5215.75
1038,Highest Activity,red,4,1,210,18,0.15,0,1.797690e+308,0.169399,0.19375,0.000000,671.284,1167.07,3824.65,871.400,1406.88,4548.00


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1040,Highest Activity,red,4,2,203,0,0.35,0,1.797690e+308,0.158470,0.16875,0.086957,616.851,1193.43,3556.05,1051.470,1582.38,4048.75
1044,Highest Activity,red,4,2,203,4,0.35,0,0.000000e+00,0.158470,0.16875,0.086957,623.365,1194.59,3564.30,1046.000,1587.00,4048.75
1045,Highest Activity,red,4,2,203,5,0.35,0,0.000000e+00,0.158470,0.16875,0.086957,618.595,1188.42,3555.20,1037.130,1585.38,4048.75
1052,Highest Activity,red,4,2,203,12,0.35,0,0.000000e+00,0.158470,0.16875,0.086957,618.162,1190.88,3551.00,1067.400,1585.25,4031.25
1056,Highest Activity,red,4,2,203,16,0.35,0,3.600000e-01,0.169399,0.18750,0.043478,649.946,1298.83,3589.49,964.467,1350.88,5259.50
1058,Highest Activity,red,4,2,203,18,0.35,0,1.797690e+308,0.174863,0.20000,0.000000,668.257,1276.14,3896.10,941.467,1331.88,4959.75
1059,Highest Activity,red,4,2,203,19,0.35,0,1.797690e+308,0.174863,0.20000,0.000000,666.892,1284.30,3518.11,910.600,1274.38,4705.12


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1060,Highest Activity,red,4,3,45,0,0.05,0,1.797690e+308,0.15847,0.18125,0.0,632.068,1130.44,3567.23,781.333,1565.25,5410.5


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1080,Highest Activity,red,4,4,124,0,0.05,0,1.797690e+308,0.153005,0.175,0.0,649.554,1161.51,4001.44,930.067,1354.88,4844.0


...

,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1120,Highest Activity,red,4,6,16,0,0.05,0,1.797690e+308,0.180328,0.20625,0.0,661.757,1191.16,3820.39,975.067,1317.88,4309.38


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1140,Highest Activity,red,4,7,273,0,0.15,0,1.797690e+308,0.153005,0.16250,0.086957,638.973,1098.40,3552.29,942.867,1383.50,4642.38
1142,Highest Activity,red,4,7,273,2,0.15,0,1.797690e+308,0.147541,0.16875,0.000000,649.851,1108.91,3399.22,914.000,1490.00,4247.00
1143,Highest Activity,red,4,7,273,3,0.15,0,1.797690e+308,0.147541,0.16875,0.000000,658.635,1104.76,3446.88,912.600,1484.62,4471.50


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1160,Highest Activity,red,4,8,225,0,0.2,0,1.797690e+308,0.153005,0.15625,0.130435,596.676,1193.83,3661.59,955.933,1313.62,4805.88
1161,Highest Activity,red,4,8,225,1,0.2,0,1.797690e+308,0.153005,0.15625,0.130435,601.189,1181.24,3686.49,1043.800,1313.62,4263.88
1162,Highest Activity,red,4,8,225,2,0.2,0,1.797690e+308,0.163934,0.18750,0.000000,616.986,1142.87,3613.84,1014.670,1419.88,4670.00
1164,Highest Activity,red,4,8,225,4,0.2,0,1.444440e+00,0.153005,0.16875,0.043478,601.649,1184.06,3629.20,1001.000,1420.12,4126.25


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1180,Highest Activity,red,4,9,155,0,0.05,0,1.797690e+308,0.163934,0.1875,0.0,649.811,1219.44,3665.4,877.467,1535.75,4699.12


[1000, 1020, 1040, 1060, 1080, 1100, 1120, 1140, 1160, 1180]


In [13]:
best_individuals_indices_lowest = [
    [0, 20, 40, 60, 80, 102, 120, 142, 162, 180], # best: 20
    
    [200, 238, 240, 260, 280, 300, 320, 340, 360, 380], #  best: 200
]


In [14]:
best_individuals_indices_average = [
    [400, 420, 440, 460, 480, 500, 520, 540, 560, 580], # best: 440
    
    [600, 620, 640, 660, 680, 700, 720, 740, 760, 780], #  best: 720
]


In [15]:
best_individuals_indices_highest = [
    [800, 822, 840, 860, 880, 924, 920, 944, 963, 984], # best: 822
    
    [1004, 1036, 1040, 1060, 1080, 1100, 1120, 1140, 1164, 1180], #  best: 1140
]

In [22]:
# DataFrame of best individuals from each seed
best_individuals_df = dataframe.loc[best_individuals_indices_highest[1]]

IPython.display.display(best_individuals_df)

# Find the overall best individual across all seeds
overall_best_individual = best_individuals_df.loc[1140]
overall_best_seed = overall_best_individual["seed"]
overall_best_fitness = overall_best_individual["percentage_violations"]
overall_best_vio_urban = overall_best_individual["percentage_violations_urban"]
overall_best_vio_rural = overall_best_individual["percentage_violations_rural"]
overall_best_individual_generations = overall_best_individual["generation"]

# Total amount of generations: count all unique generation numbers across all seeds
total_generations = filtered_df["generation"].nunique()
std_dev_fitness = best_individuals_df["percentage_violations"].std()

# Average fitness and average number of generations per seed
average_fitness_per_seed = best_individuals_df["percentage_violations"].mean()
average_generations_per_seed = best_individuals_df["generation"].mean()

print(f"Overall Best: Seed: {overall_best_seed}")
print(f"Overall Best: Fitness: {overall_best_fitness * 100:.2f}%")
print(f"Overall Best: Vio Urban: {overall_best_vio_urban * 100:.2f}%")
print(f"Overall Best: Vio Rural: {overall_best_vio_rural * 100:.2f}%")
print(f"Overall Best: Total Generations: {total_generations}")
print(f"Average Fitness per Seed: {average_fitness_per_seed * 100:.2f}%")
print(f"Average Generations per Seed: {average_generations_per_seed}")
print(f"Standard Deviation of Fitness: {std_dev_fitness * 100:.2f}%")


,day,color,ts,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
1004,Highest Activity,red,4,0,182,4,0.15,0,1.775000e+00,0.174863,0.18750,0.086957,629.757,1156.17,3660.17,990.133,1779.75,4574.38
1036,Highest Activity,red,4,1,210,16,0.15,0,4.464290e-01,0.158470,0.17500,0.043478,622.419,1235.50,3666.45,979.733,1620.12,5215.75
1040,Highest Activity,red,4,2,203,0,0.35,0,1.797690e+308,0.158470,0.16875,0.086957,616.851,1193.43,3556.05,1051.470,1582.38,4048.75
1060,Highest Activity,red,4,3,45,0,0.05,0,1.797690e+308,0.158470,0.18125,0.000000,632.068,1130.44,3567.23,781.333,1565.25,5410.50
1080,Highest Activity,red,4,4,124,0,0.05,0,1.797690e+308,0.153005,0.17500,0.000000,649.554,1161.51,4001.44,930.067,1354.88,4844.00
1100,Highest Activity,red,4,5,53,0,0.05,0,1.797690e+308,0.163934,0.18750,0.000000,633.149,1144.70,3825.77,875.400,1428.62,5102.50
1120,Highest Activity,red,4,6,16,0,0.05,0,1.797690e+308,0.180328,0.20625,0.000000,661.757,1191.16,3820.39,975.067,1317.88,4309.38
1140,Highest Activity,red,4,7,273,0,0.15,0,1.797690e+308,0.153005,0.16250,0.086957,638.973,1098.40,3552.29,942.867,1383.50,4642.38
1164,Highest Activity,red,4,8,225,4,0.20,0,1.444440e+00,0.153005,0.16875,0.043478,601.649,1184.06,3629.20,1001.000,1420.12,4126.25
1180,Highest Activity,red,4,9,155,0,0.05,0,1.797690e+308,0.163934,0.18750,0.000000,649.811,1219.44,3665.40,877.467,1535.75,4699.12


Overall Best: Seed: 7
Overall Best: Fitness: 15.30%
Overall Best: Vio Urban: 16.25%
Overall Best: Vio Rural: 8.70%
Overall Best: Total Generations: 1
Average Fitness per Seed: 16.17%
Average Generations per Seed: 148.6
Standard Deviation of Fitness: 0.94%
